In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import h5py
import scipy
from PIL import Image
from scipy import ndimage

%matplotlib inline

In [2]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

plt.imshow(train_set_x_orig[4])
print(classes[np.squeeze(train_set_y[:,4])].decode('utf-8'))

m_train = train_set_y.shape[1]
m_test = test_set_y.shape[1]
num_px = train_set_x_orig.shape[1]

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

In [4]:
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T
train_set_x = train_set_x_flatten / 255.
test_set_x = test_set_x_flatten / 255.

print ("train_set_x_flatten shape: " + str(train_set_x.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x[0:5,0]))

train_set_x_flatten shape: (12288, 209)
train_set_y shape: (1, 209)
test_set_x_flatten shape: (12288, 50)
test_set_y shape: (1, 50)
sanity check after reshaping: [0.06666667 0.12156863 0.21960784 0.08627451 0.12941176]


In [70]:
def sigmoid(x):
    return (1/(1 + np.exp(-x)))

def sigmoid_derivative(x):
    f = sigmoid (x)
    return (f*(1-f))

In [110]:


class NeuralNetwork:
    def __init__(self, x, y):
        self.layersize  = 100
        self.input      = x
        self.num_data   = self.input.shape[0]
        self.weights1   = np.zeros((self.input.shape[1],self.layersize)) 
        self.bias1      = np.zeros(self.layersize)
        self.weights2   = np.zeros((self.layersize,1))
        self.bias2      = np.zeros(1)
        self.y          = y
        self.output     = np.zeros(self.y.shape)

    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1)+self.bias1)
        self.output = sigmoid(np.dot(self.layer1, self.weights2)+self.bias2)

    def backprop(self):
        # application of the chain rule to find derivative of the loss function with respect to weights2 and weights1
        
       
        self.d_weights2 = (1/self.num_data) * np.dot(self.layer1.T, (2*(self.y - self.output) * sigmoid_derivative(self.output)))
        
    

        self.d_weights1 = (1/self.num_data) * np.dot(self.input.T,  (np.dot(2*(self.y - self.output) * sigmoid_derivative(self.output), self.weights2.T) * sigmoid_derivative(self.layer1)))

        # update the weights with the derivative (slope) of the loss function
        self.weights1 += self.d_weights1
  
        self.weights2 += self.d_weights2

data_raw = [[0,0,1,0],[0,1,1,1],[1,0,1,1],[1,1,1,0]]
data = pd.DataFrame(data_raw, columns=["x1","x2","x3","y"])
NN = NeuralNetwork(data[["x1","x2","x3"]],data[["y"]])

In [111]:
NN = NeuralNetwork(train_set_x.T,train_set_y.T)

In [120]:
for i in range(1000):
    NN.feedforward()
    NN.backprop()

In [127]:
sigmoid_derivative(NN.output)

array([[0.23500371],
       [0.25      ],
       [0.23500371],
       [0.25      ],
       [0.25      ],
       [0.24194468],
       [0.25      ],
       [0.23500371],
       [0.25      ],
       [0.25      ],
       [0.23500371],
       [0.23500371],
       [0.23500396],
       [0.23500371],
       [0.23500386],
       [0.25      ],
       [0.25      ],
       [0.24875435],
       [0.23501901],
       [0.23500371],
       [0.25      ],
       [0.25      ],
       [0.23500371],
       [0.24503272],
       [0.23500371],
       [0.23500371],
       [0.25      ],
       [0.23500371],
       [0.24989765],
       [0.23500371],
       [0.25      ],
       [0.24996199],
       [0.23500371],
       [0.23500493],
       [0.24999965],
       [0.23620783],
       [0.25      ],
       [0.23556243],
       [0.23500371],
       [0.23500371],
       [0.25      ],
       [0.24997515],
       [0.23500371],
       [0.24999917],
       [0.25      ],
       [0.23500417],
       [0.23519087],
       [0.242

In [152]:
train_set_y[0]

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)